<a href="https://colab.research.google.com/github/camilla-scandola/project-shark-attack/blob/main/quest_sharkatack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pycountry

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import re

In [6]:
df = pd.read_excel('GSAF5.xls')

In [7]:
pd.set_option('display.max_columns', None)
df.head()


,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,16th August 2025,2025.0,Provoked,USA,Florida,Cayo Costa Boca Grande,Fishing,Shawn Meuse,M,?,Laceration to right leg below the knee,N,1055 hrs,Lemon shark 1.8 m (6ft),Johannes Marchand: Kevin McMurray Trackingshar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,Brad Ross,M,?,None sustained board severly damaged,N,0730hrs,5m (16.5ft) Great White,Bob Myatt GSAF The Guardian: 9 News: ABS News:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,Not stated,M,63,Severe injuries no detail,N,1300hrs,Undetermined,Ralph Collier GSAF and Kevin MCMurray Tracking...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7th August,2025.0,Unprovoked,Australia,NSW,Tathra Beach,Surfing,Bowie Daley,M,9,None sustained board severely damaged,N,1630hrs,Suspected Great White,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1st August,2025.0,Unprovoked,Puerto Rico,Carolina,Carolina Beach,Wading,Eleonora Boi,F,39,Bite to thigh area,N,Not stated,Undetermined,Kevin McMurray Trackingsharks.com: NY Post,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#overview of the dataset

missing_country = int(df['Country'].isna().sum())
print(missing_country)

missing_state = int(df['State'].isna().sum())
print(missing_state)

missing_location = int(df['Location'].isna().sum())
print(missing_location)

print(df['Country'].tail())

#suggestion: drop all the null values under country (because they won't make a big difference in the analysis, being 50 out of 7041)
#df['Country'].dropna()

50
485
567
7037             AUSTRALIA
7038             AUSTRALIA
7039                   USA
7040                PANAMA
7041    CEYLON (SRI LANKA)
Name: Country, dtype: object


In [9]:
#suggestion: the number of missing values in states and locations is a lot more significant, so I'd replace them with undefined for the time being
df['State'].fillna('undefined')
df['Location'].fillna('undefined')

0                    Cayo Costa Boca Grande
1                            Cabarita Beach
2       North of Grand Bahama near Freeport
3                             Tathra Beach 
4                            Carolina Beach
                       ...                 
7037                            Roebuck Bay
7038                              undefined
7039                         Ocracoke Inlet
7040                   Panama Bay 8ºN, 79ºW
7041    Below the English fort, Trincomalee
Name: Location, Length: 7042, dtype: object

In [12]:
#if needed, imported pycountry to check all standard country names + 

import pycountry

df['Country_cleaned'] = df['Country'].astype(str).str.strip().str.lower().str.title()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    
    print(df['Country_cleaned'].value_counts())

Country_cleaned
Usa                                      2570
Australia                                1507
South Africa                              599
New Zealand                               146
Bahamas                                   141
Papua New Guinea                          136
Brazil                                    122
Mexico                                    107
Italy                                      72
Fiji                                       70
New Caledonia                              66
Philippines                                65
Reunion                                    60
Egypt                                      53
Mozambique                                 50
Nan                                        50
Cuba                                       49
Spain                                      47
India                                      41
French Polynesia                           39
Japan                                      36
Croatia           

In [13]:
# Mapping of common mismatches to ISO pycountry names
replacements = {
    # Abbreviations & Short forms
    "Usa": "United States",
    "UK": "United Kingdom",
    "UAE": "United Arab Emirates",

    # Alternative spellings
    "South Korea": "Korea, Republic of",
    "North Korea": "Korea, Democratic People's Republic of",
    "Russia": "Russian Federation",
    "Iran": "Iran, Islamic Republic of",
    "Syria": "Syrian Arab Republic",
    "Tanzania": "Tanzania, United Republic of",
    "Vietnam": "Viet Nam",
    "Turkey": "Türkiye",
    "Bolivia": "Bolivia, Plurinational State of",
    "Venezuela": "Venezuela, Bolivarian Republic of",
    "Moldova": "Moldova, Republic of",
    "Laos": "Lao People's Democratic Republic",
    "Palestinian Territories": "Palestine, State of",
    "Micronesia": "Micronesia, Federated States of",
    "Macedonia": "North Macedonia",

    # Historical names
    "Burma": "Myanmar",
    "Ceylon": "Sri Lanka",
    "Ceylon (Sri Lanka)": "Sri Lanka",
    "Western Samoa": "Samoa",
    "Zaire": "Congo, The Democratic Republic of the",

    # Variations / territories that map to countries
    "Curacao": "Curaçao",
    "St. Martin": "Saint Martin (French part)",
    "St Martin": "Saint Martin (French part)",
    "St. Maartin": "Sint Maarten (Dutch part)",
    "Reunion": "Réunion",
    "Reunion Island": "Réunion",
    "Bahrein": "Bahrain",
    "Cape Verde": "Cabo Verde",
    "Taiwan": "Taiwan, Province of China",
    "Maldive Islands": "Maldives",
}

df["Country_cleaned"] = df["Country_cleaned"].replace(replacements)

df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22,Country_cleaned
0,16th August 2025,2025.0,Provoked,USA,Florida,Cayo Costa Boca Grande,Fishing,Shawn Meuse,M,?,Laceration to right leg below the knee,N,1055 hrs,Lemon shark 1.8 m (6ft),Johannes Marchand: Kevin McMurray Trackingshar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States
1,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,Brad Ross,M,?,None sustained board severly damaged,N,0730hrs,5m (16.5ft) Great White,Bob Myatt GSAF The Guardian: 9 News: ABS News:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Australia
2,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,Not stated,M,63,Severe injuries no detail,N,1300hrs,Undetermined,Ralph Collier GSAF and Kevin MCMurray Tracking...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bahamas
3,7th August,2025.0,Unprovoked,Australia,NSW,Tathra Beach,Surfing,Bowie Daley,M,9,None sustained board severely damaged,N,1630hrs,Suspected Great White,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Australia
4,1st August,2025.0,Unprovoked,Puerto Rico,Carolina,Carolina Beach,Wading,Eleonora Boi,F,39,Bite to thigh area,N,Not stated,Undetermined,Kevin McMurray Trackingsharks.com: NY Post,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Puerto Rico


In [18]:
pycountry_names = {c.name for c in pycountry.countries}
df = df[df['Country_cleaned'].isin(pycountry_names)]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df['Country_cleaned'].value_counts())

Country_cleaned
United States                        2570
Australia                            1507
South Africa                          599
New Zealand                           146
Bahamas                               141
Papua New Guinea                      136
Brazil                                122
Mexico                                107
Italy                                  72
Fiji                                   70
New Caledonia                          66
Philippines                            65
Réunion                                65
Egypt                                  53
Mozambique                             50
Cuba                                   49
Spain                                  47
India                                  41
French Polynesia                       39
Japan                                  36
Croatia                                35
Jamaica                                32
Panama                                 32
Solomon Islands   

In [ ]:
#Columns Fatal + Deaths
#checking all unique values in Fatal, to understand how I can better clean it
print(df['Fatal Y/N'].unique())

In [ ]:
#cleaning Fatal and printing unique values
df['Fatal Y/N cleaned'] = df['Fatal Y/N'].astype(str).str.strip().str.upper().apply(lambda x: re.sub(r'[^A-Z]', '', x))
print(df['Fatal Y/N cleaned'].unique())

#add column "Death" and assign a numeric value to Fatal, yes or no
deaths_count = {'N': 0, 'Y': 1}
df['Death'] = df['Fatal Y/N cleaned'].map(deaths_count)
deaths_count = int(df['Death'].sum())
print(deaths_count)

In [ ]:
#checking all unique values in Fatal after clean-up
counts = df['Fatal Y/N cleaned'].value_counts(dropna=False)
print(counts)

#all data that is neither Y nor N, is replaced with NaN in the transition from numeric values
counts = df['Death'].value_counts(dropna=False)
print(counts)

In [ ]:
df.head()